In [2]:
import torch
from nuclr.data import prepare_nuclear_data
from nuclr.config import config
import tqdm

In [3]:
data = prepare_nuclear_data(config)

In [5]:
X = data.X.clone()
X[:, 1:] += data.vocab_size[0]
X[:, 2:] += data.vocab_size[1]
y = data.y

In [61]:
d_model = 32
torch.manual_seed(0)
model = torch.nn.Sequential(
    torch.nn.Embedding(sum(data.vocab_size), d_model),
    torch.nn.Flatten(),
    torch.nn.Linear(d_model * 3, d_model),
    torch.nn.ReLU(),
    torch.nn.Linear(d_model, d_model),
    torch.nn.ReLU(),
    torch.nn.Linear(d_model, 1),
).to(config.DEV)

In [62]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
mask = ~y.isnan()

for i in (pbar:=tqdm.trange(2)):
    optimizer.zero_grad()
    loss = (model(X) - y) ** 2
    loss = loss[mask]
    print("masked", loss)
    loss = loss.mean()
    print("loss", loss)
    loss.backward()
    print("grad", model[0].weight.grad)
    optimizer.step()
    pbar.set_description("Loss: {:.4f}".format(loss.item()))

Loss: nan: 100%|██████████| 2/2 [00:00<00:00, 183.31it/s]

masked tensor([0.2387, 0.2045, 0.0246,  ..., 0.1680, 0.1131, 0.0888], device='cuda:0',
       grad_fn=<IndexBackward0>)
loss tensor(0.3227, device='cuda:0', grad_fn=<MeanBackward0>)
grad tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0')
masked tensor([nan, nan, nan,  ..., nan, nan, nan], device='cuda:0',
       grad_fn=<IndexBackward0>)
loss tensor(nan, device='cuda:0', grad_fn=<MeanBackward0>)
grad tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0')
